# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
![Passed2](passed2.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [4]:
# Import
import glob

import numpy as np
import scipy as sp
from scipy.io import loadmat
from matplotlib import pyplot as plt
import scipy.signal
import pandas as pd
import matplotlib.ticker as plticker


%matplotlib inline

In [5]:
def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))

def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
        # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)



In [6]:
def slice_signals(signals_set, window_s, overlap_s, fs):
    """
        This function slice signals comming from ppg and accelerometer based on the window and overlap selected
        
        @input Args
            - signals_set: array containing ppg, accx, accy, accz signals to be sliced. Signals must be of same size
            - window_s: time frame in [s] to which we will slice our signals
            - overlap_s: time frame in [s] to which we will use to set the stride of the window.
            - fs: Sample rate of the signals
        @output return
            - sliced_signals: ppg, accx,accy,accz sliced signals.
            
    """
    window_sample = int(window_s * fs) # convert window to sample window
    overlap_sample = int(overlap_s * fs) # convert overlap to sample overlap
    window_stride = window_sample-overlap_sample
    sliced_signals = []
    for i in range(0, signals_set.shape[1] - window_sample, window_stride):
        signal_slice = signals_set[:,i:i+window_sample]
        sliced_signals.append(signal_slice)
    return np.array(sliced_signals)

def BandpassFilter(signal, pass_band, fs):
    """Bandpass Filter.
    
    @input Args
        signal: (np.array) The input signal
        pass_band: (tuple) The pass band. Frequency components outside 
            the two elements in the tuple will be removed.
        fs: (number) The sampling rate of <signal>
        
    @output returns
        (np.array) The filtered signal
    """
    b, a = sp.signal.butter(3, pass_band, btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def spectrogram_slices(signal, window_s, overlap_s, fs, plot=False):
    """
        Computes the spectrogram magnitude of a signal and returns the fft amplitudes and frequencies of it.
        
        @input Args
            signal: (numpy array) signal we want to get the spectrogram from.
            window_s: (float) time frame of the window in seconds
            overlap_s: (float) time frame of the overlap between windows in seconds
            fs: (float) sample rate in Hz
            plot: (Boolean) defines if we want to plot or not plot the spectrogram
        
        @output returns
            spec_amp: (2D numpy array) Amplitudes of the spectrogram per window
            freqs: (numpy array) Frequencies of the spectrogram fow all windows
            
    """
    
    NFFT = fs*window_s # window_s secs to samples
    overlap = fs*overlap_s # overlap_s secs to samples
    time = np.array(range(0,len(signal)))/fs/60 # time in minutes
    if plot:
        #fig=plt.figure(figsize=(15,8))
        spec_amp,  freqs, t, cax= plt.specgram(signal, Fs=fs, NFFT=NFFT, noverlap=overlap, 
                                              scale='linear', mode='magnitude', xextent=(0,time[-1]))
        cbar = plt.colorbar();
        cbar.ax.set_ylabel('Amplitude', rotation=90, labelpad=15, fontsize=20);
        plt.ylim(0,10)
        plt.xlabel('Time [min]', fontsize=20);
        plt.ylabel('frequencies [Hz]', fontsize=20);
        print(f"Shape of the amplitudes in spectrogram {spec_amp.shape} and number of frequencies: {len(freqs)}")
    else:
        fig=plt.figure()
        spec_amp,  freqs, _, _= plt.specgram(signal, Fs=fs, NFFT=NFFT, noverlap=overlap, 
                                              scale='linear', mode='magnitude', xextent=(0,time[-1]))
        plt.close(fig);
    return spec_amp, freqs

def acc_magnitude(accx, accy, accz): 
    """
        Takes accx, accy, accz and returns the magnitude
    """
    return np.sqrt(accx**2+accy**2+accz**2)

def calculate_confidence(amplitudes, frequencies, est_BPM, meas_uncertainty_BPM=0.5):
    """
        Calculates confidence of the measurement based on the amplitude power of the signal
        
        @input Args
            amplitudes: 1D numpy array containing all amplitudes of the signal
            frequencies: 1D numpy array containing all frequencies of the signal, must be same size as amplitudes (Hz)
            est_BPM: Estimated frequency in BPM (float number)
            meas_uncertainty: measurement uncertainty of our estimation in BPM, by default is set 
                              to 0.5 (float number)
        
        @output returns
            confidence: float number that represents how confident our measurement is
    """
    est_Hz = est_BPM/60
    meas_uncertainty_Hz = meas_uncertainty_BPM/60
    uncertainty_window = (frequencies>=est_Hz-meas_uncertainty_Hz) & (frequencies<=est_Hz+meas_uncertainty_Hz)
    confidence = np.sum(amplitudes[uncertainty_window])/np.sum(amplitudes)
    return confidence

def estimation_error(est_BPM, label_BPM):
    """
        Calculates the absolute error of certain estimation
    """
    return np.abs(est_BPM-label_BPM)

def Hz2BPM(value_Hz):
    """
        Converts a value in Hz(hertz) to BPM(beats per minute)
    """
    return value_Hz*60.0

In [7]:
def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Run Pulse Rate Algorithm estimator based on ppg and acc signal
       
    @input Args:
        data_fl: (string) Path to data file (.mat)
        ref_fl: (string) Path to reference data file (.mat)
        
    @ Output Returns:
        errors: (np.array) Error scores
        confidence: (np.array) Confidence rates
    """
    
    ## Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    # getting ground truth 
    labels = loadmat(ref_fl)['BPM0']
    
    ## Parameters
    # sample rate
    fs = 125 # given by the dataset provider in Hz
    # define time vector
    time = np.array(range(0,ppg.shape[0]))/fs/60 # time in minutes
    # define tolerance to check for similarity between ppg and acc in frequency domain
    tolerance = 0.2 #Hz
        
    ## Compute pulse rate estimates and estimation confidence.

    # Filtering signals
    filtered_ppg = BandpassFilter(ppg, pass_band=(0.667,4), fs=fs)
    acc_magnitude_signal = acc_magnitude(accx, accy, accz)
    filtered_acc = BandpassFilter(acc_magnitude_signal, pass_band=(0.667,4), fs=fs)
    
    # Compute the spectrogram of both signals
    ampls_pgg, freqs_ppg = spectrogram_slices(filtered_ppg, window_s=8, overlap_s=6, fs=fs, plot=False)
    ampls_acc, freqs_acc = spectrogram_slices(filtered_acc, window_s=8, overlap_s=6, fs=fs, plot=False)
    
    
    errors=[]
    confidence=[]
    
    for window_i in range(ampls_pgg.shape[1]):

        # Getting the window i amplitudes
        ampl_ppg_i = ampls_pgg[:,window_i]
        ampl_acc_i = ampls_acc[:,window_i]
        
        # Find peaks on frequency domain of both ppg and acc       
        pks_ppg_i = sp.signal.find_peaks(ampl_ppg_i, prominence=0.5*np.max(ampl_ppg_i))[0]
        pks_acc_i = sp.signal.find_peaks(ampl_acc_i, prominence=0.1*np.max(ampl_acc_i))[0]
        
        # Getting top frequencies
        ppg_top_freqs = freqs_ppg[pks_ppg_i]
        acc_top_freqs = freqs_acc[pks_acc_i]

        try:
            est_heart_rate = ppg_top_freqs[0] #won't change if arm motion is equal to heart beat frequency
        except:
            break
        for ppg_top_freq in ppg_top_freqs:
            if not any(abs(ppg_top_freq-acc_top_freqs)<tolerance):
                est_heart_rate = ppg_top_freq
                break
        error_est_i = estimation_error(Hz2BPM(est_heart_rate), labels[window_i])
        confidence_i = calculate_confidence(ampl_ppg_i, freqs_ppg, Hz2BPM(est_heart_rate), meas_uncertainty_BPM=0.5)

        errors.append(error_est_i)
        confidence.append(confidence_i)

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    errors, confidence = np.mean(errors), np.mean(confidence)  # Dummy placeholders. Remove
    return errors, confidence